# Set Environment (No Split, No Double Down)

In [1]:
import gym
from gym import spaces
from gym.utils import seeding
import random

# Full deck with distinct face cards
CARDS = [1, 2, 3, 4, 5, 6, 7, 8, 9, '10', 'J', 'Q', 'K'] * 4

def card_value(card):
    return 10 if card in ['10', 'J', 'Q', 'K'] else card

def draw_card(deck):
    return deck.pop()

def draw_hand(deck):
    return [draw_card(deck), draw_card(deck)]

def usable_ace(hand):
    return 1 in hand and sum(card_value(c) for c in hand) + 10 <= 21

def sum_hand(hand):
    total = sum(card_value(c) for c in hand)
    return total + 10 if usable_ace(hand) else total

def is_bust(hand):
    return sum_hand(hand) > 21

def score(hand):
    return 0 if is_bust(hand) else sum_hand(hand)

def is_natural(hand):
    return set(hand) == {1, '10'} or set(hand) == {1, 'J'} or set(hand) == {1, 'Q'} or set(hand) == {1, 'K'}

class BlackjackEnv(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(self, numdecks=4, natural=True):
        super().__init__()
        self.action_space = spaces.Discrete(2)  # 0: Stick, 1: Hit
        self.observation_space = spaces.Tuple((
            spaces.Tuple((spaces.Discrete(32), spaces.Discrete(32))),  # Player hand (2 cards)
            spaces.Discrete(11),  # Dealer's showing card
            spaces.Discrete(2)    # Usable ace
        ))

        self.natural = natural
        self.numdecks = numdecks
        self.decks = CARDS * self.numdecks
        random.shuffle(self.decks)
        self.seed()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        random.seed(seed)
        return [seed]

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.seed(seed)

        if self._deck_is_out():
            self.decks = CARDS * self.numdecks
            random.shuffle(self.decks)

        self.dealer = draw_hand(self.decks)
        first_hand = draw_hand(self.decks)
        self.hands = [first_hand]
        self.current_hand = 0
        self.actionstaken = 0
        self.hand_results = []
        return self._get_obs()

    def step(self, action):
        assert self.action_space.contains(action), f"Invalid action: {action}"
        if self._deck_is_out():
            self.decks = CARDS * self.numdecks
            random.shuffle(self.decks)

        done = False
        reward = 0
        hand = self.hands[self.current_hand]

        if action == 0:  # Stick
            self._finalize_current_hand()

        elif action == 1:  # Hit
            hand.append(draw_card(self.decks))
            if is_bust(hand):
                self.hand_results.append(-1)
                self._advance_hand()

        self.actionstaken += 1

        if self.current_hand >= len(self.hands):
            while sum_hand(self.dealer) < 17:
                self.dealer.append(draw_card(self.decks))

            if len(self.hand_results) < len(self.hands):
                self._finalize_current_hand()

            reward = sum(self.hand_results)
            done = True

        return self._get_obs(), reward, done, {}

    def _finalize_current_hand(self):
        hand = self.hands[self.current_hand]
        player_score = score(hand)
        dealer_score = score(self.dealer)
        result = float(player_score > dealer_score) - float(player_score < dealer_score)
        if is_natural(hand) and result == 1 and self.natural:
            result = 1.5
        self.hand_results.append(result)
        self._advance_hand()

    def _advance_hand(self):
        self.current_hand += 1
        self.actionstaken = 0

    def _get_obs(self):
        if self.current_hand >= len(self.hands):
            return ((0, 0), card_value(self.dealer[0]), 0)

        hand = self.hands[self.current_hand]
        padded = hand[:2] + [0] * (2 - len(hand))
        return (
            tuple(card_value(c) if c != 0 else 0 for c in padded[:2]),
            card_value(self.dealer[0]),
            int(usable_ace(hand))
        )

    def _deck_is_out(self):
        return len(self.decks) < self.numdecks * len(CARDS) * 0.1

# Set the Simple DQN Model

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque
import copy
import os

# Define the Q-network
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    
    def forward(self, x):
        return self.fc(x)
    
def preprocess_state(state):
    """
    Converts ((card1, card2), dealer_card, usable_ace) => [player_sum, dealer_card, usable_ace]
    """
    player_cards, dealer_card, usable_ace = state
    player_sum = sum(player_cards)
    return np.array([player_sum, dealer_card, usable_ace], dtype=np.float32)

# === Action Selection: Epsilon-Greedy ===
def select_action(state, q_network, epsilon, action_space):
    if random.random() < epsilon:
        return action_space.sample()
    with torch.no_grad():
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        q_values = q_network(state_tensor)
        return q_values.argmax().item()
    
def select_action(state, q_network, epsilon, action_space):
    if random.random() < epsilon:
        return action_space.sample()
    with torch.no_grad():
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        q_values = q_network(state_tensor)
        return q_values.argmax().item()

def train_dqn(env, n_episodes=5000, gamma=0.99, lr=1e-3, batch_size=64,
              epsilon_start=1.0, epsilon_end=0.1, epsilon_decay=0.995,
              model_save_path='best_blackjack_dqn.pth'):

    input_dim = 3  # [player_sum, dealer_card, usable_ace]
    output_dim = env.action_space.n

    q_network = QNetwork(input_dim, output_dim)
    optimizer = optim.Adam(q_network.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    replay_buffer = deque(maxlen=10000)
    epsilon = epsilon_start
    losses = []

    best_model = None
    best_avg_loss = float('inf')
    loss_window = []

    for episode in range(n_episodes):
        state = env.reset()
        state = preprocess_state(state)
        done = False

        while not done:
            action = select_action(state, q_network, epsilon, env.action_space)
            next_state, reward, done, _ = env.step(action)
            next_state = preprocess_state(next_state)

            replay_buffer.append((state, action, reward, next_state, done))
            state = next_state

            if len(replay_buffer) >= batch_size:
                batch = random.sample(replay_buffer, batch_size)
                states, actions, rewards, next_states, dones = zip(*batch)

                states_tensor = torch.FloatTensor(np.array(states))
                actions_tensor = torch.LongTensor(actions).unsqueeze(1)
                rewards_tensor = torch.FloatTensor(rewards).unsqueeze(1)
                next_states_tensor = torch.FloatTensor(np.array(next_states))
                dones_tensor = torch.BoolTensor(dones).unsqueeze(1)

                with torch.no_grad():
                    next_q_values = q_network(next_states_tensor).max(1, keepdim=True)[0]
                    targets = rewards_tensor + gamma * next_q_values * (~dones_tensor)

                q_values = q_network(states_tensor).gather(1, actions_tensor)

                loss = loss_fn(q_values, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                losses.append(loss.item())
                loss_window.append(loss.item())
                if len(loss_window) > 100:
                    loss_window.pop(0)

        # Decay epsilon
        epsilon = max(epsilon_end, epsilon * epsilon_decay)

        # Save best model
        if len(loss_window) == 100:
            avg_loss = np.mean(loss_window)
            if avg_loss < best_avg_loss:
                best_avg_loss = avg_loss
                best_model = copy.deepcopy(q_network)
                torch.save({
                    'model_state_dict': best_model.state_dict(),
                    'avg_loss': best_avg_loss,
                    'episode': episode + 1
                }, model_save_path)
                print(f"✅ Best model saved at episode {episode+1} | Avg Loss: {best_avg_loss:.4f}")

        if (episode + 1) % 1000 == 0:
            print(f"Episode {episode+1} | Epsilon: {epsilon:.4f} | Recent Loss: {losses[-1]:.4f}")

    return best_model if best_model else q_network, losses

In [ ]:
# Train models for different deck counts
print("Training DQN models for different deck counts ===")
dqn_models = {}

for num_decks in range(1, 7):
    print(f"\n=== Training model for {num_decks} deck(s) ===")
    env = BlackjackEnv(numdecks=num_decks, natural=True)
    model_save_path = f"blackjack_dqn_decks_{num_decks}.pth"
    model, _ = train_dqn(env, n_episodes=10000, model_save_path=model_save_path)
    dqn_models[num_decks] = model
    print(f"Completed training for {num_decks} deck(s)")

print("All models trained successfully!")

In [6]:
import pandas as pd
import random
import torch
import numpy as np
import os
import matplotlib.pyplot as plt

# Function to load a single DQN model
def load_dqn_model(num_decks):
    """
    Load a single DQN model for the specified number of decks
    """
    model_path = f"blackjack_dqn_decks_{num_decks}.pth"
    
    if os.path.exists(model_path):
        # Create a new model with the correct dimensions
        input_dim = 3  # [player_sum, dealer_card, usable_ace]
        output_dim = 2  # [stick, hit]
        model = QNetwork(input_dim, output_dim)
        
        # Load the saved weights
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()  # Set to evaluation mode
        
        print(f"  Successfully loaded model from {model_path}")
        return model
    else:
        print(f"  ERROR: Model file not found at {model_path}")
        return None

# Helper to preprocess state
def preprocess_state(state):
    player_cards, dealer_card, usable_ace = state
    player_sum = sum(player_cards)
    return np.array([player_sum, dealer_card, usable_ace], dtype=np.float32)

# Evaluate a single DQN model on a specific deck size
def evaluate_dqn_model(q_network, num_decks, num_games=10000):
    
    env = BlackjackEnv(numdecks=num_decks, natural=True)

    wins = 0
    losses = 0
    draws = 0
    total_reward = 0

    for game in range(num_games):
        obs = env.reset()
        state = preprocess_state(obs)
        done = False
        episode_reward = 0

        while not done:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0)
                action = q_network(state_tensor).argmax().item()

            next_obs, reward, done, _ = env.step(action)
            state = preprocess_state(next_obs)
            episode_reward += reward

        total_reward += episode_reward
        if episode_reward > 0:
            wins += 1
        elif episode_reward < 0:
            losses += 1
        else:
            draws += 1

    # Return results as a dictionary
    return {
        "Decks": num_decks,
        "Games": num_games,
        "Wins": wins,
        "Draws": draws,
        "Losses": losses,
        "Total Reward": round(total_reward, 4),
        "Win Rate (%)": round((wins / num_games) * 100, 4),
        "Loss Rate (%)": round((losses / num_games) * 100, 4),
        "Draw Rate (%)": round((draws / num_games) * 100, 4),
        "Average Reward": round(total_reward / num_games, 4)
    }

In [7]:
# Load models for each deck count
print("\nLoading DQN models...")
dqn_models = {}

for num_decks in range(1, 7):
    print(f"Loading model for {num_decks} deck(s)...")
    model = load_dqn_model(num_decks)
    if model is not None:
        dqn_models[num_decks] = model

# Evaluate each model and collect results
print("\nEvaluating DQN models...")
evaluation_results = []

for num_decks, model in dqn_models.items():
    print(f"Evaluating model for {num_decks} deck(s)...")
    result = evaluate_dqn_model(model, num_decks, num_games=10000)
    evaluation_results.append(result)
    print(f"  Win Rate: {result['Win Rate (%)']:.2f}%, Avg Reward: {result['Average Reward']:.4f}")

# Convert results to DataFrame
df_dqn_results = pd.DataFrame(evaluation_results)
df_dqn_results


Loading DQN models...
Loading model for 1 deck(s)...
  Successfully loaded model from blackjack_dqn_decks_1.pth
Loading model for 2 deck(s)...
  Successfully loaded model from blackjack_dqn_decks_2.pth
Loading model for 3 deck(s)...
  Successfully loaded model from blackjack_dqn_decks_3.pth
Loading model for 4 deck(s)...
  Successfully loaded model from blackjack_dqn_decks_4.pth
Loading model for 5 deck(s)...
  Successfully loaded model from blackjack_dqn_decks_5.pth
Loading model for 6 deck(s)...
  Successfully loaded model from blackjack_dqn_decks_6.pth

Evaluating DQN models...
Evaluating model for 1 deck(s)...
  Win Rate: 47.81%, Avg Reward: 0.0394
Evaluating model for 2 deck(s)...
  Win Rate: 47.49%, Avg Reward: 0.0429
Evaluating model for 3 deck(s)...
  Win Rate: 46.69%, Avg Reward: 0.0248
Evaluating model for 4 deck(s)...
  Win Rate: 47.12%, Avg Reward: 0.0328
Evaluating model for 5 deck(s)...
  Win Rate: 47.32%, Avg Reward: 0.0352
Evaluating model for 6 deck(s)...
  Win Rate: 

,Decks,Games,Wins,Draws,Losses,Total Reward,Win Rate (%),Loss Rate (%),Draw Rate (%),Average Reward
0,1,10000,4781,608,4611,394.0,47.81,46.11,6.08,0.0394
1,2,10000,4749,688,4563,429.0,47.49,45.63,6.88,0.0429
2,3,10000,4669,699,4632,248.0,46.69,46.32,6.99,0.0248
3,4,10000,4712,689,4599,328.5,47.12,45.99,6.89,0.0328
4,5,10000,4732,653,4615,351.5,47.32,46.15,6.53,0.0352
5,6,10000,4674,657,4669,230.5,46.74,46.69,6.57,0.0231


In [8]:
# Helper function to preprocess state
def preprocess_state(state):
    player_cards, dealer_card, usable_ace = state
    player_sum = sum(player_cards)
    return np.array([player_sum, dealer_card, usable_ace], dtype=np.float32)

# Define the bankroll evaluation function
def evaluate_dqn_bankroll(models, num_games=10000, max_decks=6, initial_money=100):
    """
    Evaluate DQN models with a bankroll simulation across different deck sizes
    """
    results = []

    for num_deck in range(1, max_decks + 1):
        
        env = BlackjackEnv(numdecks=num_deck, natural=True)
        q_network = models[num_deck]  # Get the specific model for this deck size
        q_network.eval()  # Set the model to evaluation mode

        money = initial_money
        wins = 0
        losses = 0
        draws = 0
        total_reward = 0
        
        # For tracking bankruptcy
        games_played = 0
        went_bankrupt = False

        for game in range(1, num_games+1):
            if money <= 0:
                went_bankrupt = True
                games_played = game - 1
                break
                
            games_played = game
            obs = env.reset()
            done = False
            
            # Bet $1
            money -= 1
            episode_reward = 0
            doubled_down = False

            while not done:
                # Process state to match training format
                state = preprocess_state(obs)
                
                with torch.no_grad():
                    state_tensor = torch.FloatTensor(state).unsqueeze(0)
                    q_values = q_network(state_tensor)
                    
                    # Get valid actions for the current state
                    valid_actions = [0, 1]  # Stick, Hit are always valid
                    
                    # Mask invalid actions
                    masked_q_values = q_values.clone()
                    for i in range(q_values.size(1)):
                        if i not in valid_actions:
                            masked_q_values[0, i] = float('-inf')
                    
                    action = torch.argmax(masked_q_values, dim=1).item()
                
                # Check if action is valid (safeguard)
                if action not in valid_actions:
                    action = 0  # Default to stick if somehow invalid
                
                # Execute the action
                try:
                    next_obs, reward, done, _ = env.step(action)
                    episode_reward += reward
                    obs = next_obs
                except Exception as e:
                    # Fallback if error
                    print(f"Error executing action {action}: {e}")
                    action = 0  # Stick
                    next_obs, reward, done, _ = env.step(action)
                    episode_reward += reward
                    obs = next_obs

            # End of episode accounting
            total_reward += episode_reward
            
            # Update wins/losses/draws and bankroll
            if episode_reward > 0:
                wins += 1
                # Calculate payout
                if doubled_down:
                    money += 4  # Win 2x the doubled bet
                else:
                    if episode_reward > 1:  # Blackjack
                        money += 2.5  # 3:2 payout
                    else:
                        money += 2  # Even money
            elif episode_reward < 0:
                losses += 1
                # Money already subtracted for bet
            else:
                draws += 1
                if doubled_down:
                    money += 2  # Get doubled bet back
                else:
                    money += 1  # Get original bet back

        # Store results
        bankruptcy_message = f"Bankrupt after {games_played} games" if went_bankrupt else "Solvent"
        
        results.append({
            "Decks": num_deck,
            "Games": games_played,
            "Wins": wins,
            "Draws": draws,
            "Losses": losses,
            "Total Reward": round(total_reward, 4),
            "Win Rate (%)": round((wins / games_played) * 100, 4) if games_played > 0 else 0,
            "Loss Rate (%)": round((losses / games_played) * 100, 4) if games_played > 0 else 0,
            "Draw Rate (%)": round((draws / games_played) * 100, 4) if games_played > 0 else 0,
            "Average Reward": round(total_reward / games_played, 4) if games_played > 0 else 0,
            "Final Money": round(money, 2),
            "Status": bankruptcy_message
        })
        
        print(f"Completed simulation for {num_deck} deck(s)")
        print(f"  Win Rate: {(wins/games_played)*100:.2f}%, Final Money: ${money:.2f}")

    return pd.DataFrame(results)

# Run the bankroll experiment
print("\nRunning DQN bankroll experiment...")
df_dqn_bankroll = evaluate_dqn_bankroll(dqn_models, num_games=10000, max_decks=6, initial_money=100)
df_dqn_bankroll


Running DQN bankroll experiment...
Completed simulation for 1 deck(s)
  Win Rate: 47.60%, Final Money: $427.00
Completed simulation for 2 deck(s)
  Win Rate: 48.29%, Final Money: $671.00
Completed simulation for 3 deck(s)
  Win Rate: 47.43%, Final Money: $478.50
Completed simulation for 4 deck(s)
  Win Rate: 46.23%, Final Money: $263.50
Completed simulation for 5 deck(s)
  Win Rate: 47.31%, Final Money: $483.50
Completed simulation for 6 deck(s)
  Win Rate: 46.92%, Final Money: $424.50


,Decks,Games,Wins,Draws,Losses,Total Reward,Win Rate (%),Loss Rate (%),Draw Rate (%),Average Reward,Final Money,Status
0,1,10000,4760,576,4664,327.0,47.60,46.64,5.76,0.0327,427.0,Solvent
1,2,10000,4829,675,4496,571.0,48.29,44.96,6.75,0.0571,671.0,Solvent
2,3,10000,4743,680,4577,378.5,47.43,45.77,6.80,0.0379,478.5,Solvent
3,4,10000,4623,707,4670,163.5,46.23,46.70,7.07,0.0163,263.5,Solvent
4,5,10000,4731,698,4571,383.5,47.31,45.71,6.98,0.0384,483.5,Solvent
5,6,10000,4692,718,4590,324.5,46.92,45.90,7.18,0.0324,424.5,Solvent


# Set PPO

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

# PPO Actor-Critic Network
class PPOActorCritic(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PPOActorCritic, self).__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.policy_head = nn.Linear(64, output_dim)
        self.value_head = nn.Linear(64, 1)

    def forward(self, x):
        shared_out = self.shared(x)
        logits = self.policy_head(shared_out)
        value = self.value_head(shared_out)
        return logits, value

# Adjusted for BlackjackEnv
def preprocess_state(state):
    """
    Convert ((card1, card2), dealer_card, usable_ace) => [player_sum, dealer_card, usable_ace]
    """
    player_cards, dealer_card, usable_ace = state
    player_sum = sum(player_cards)
    return np.array([player_sum, dealer_card, usable_ace], dtype=np.float32)

# Compute GAE
def compute_gae(rewards, values, dones, gamma=0.99, lam=0.95):
    returns = []
    advantages = []
    gae = 0
    next_value = 0

    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * next_value * (1 - dones[step]) - values[step]
        gae = delta + gamma * lam * (1 - dones[step]) * gae
        advantages.insert(0, gae)
        returns.insert(0, gae + values[step])
        next_value = values[step]

    return torch.FloatTensor(returns), torch.FloatTensor(advantages)

# PPO Training Function
def train_ppo(env, n_episodes=5000, gamma=0.99, lam=0.95, clip_eps=0.2,
              lr=3e-4, epochs=4, batch_size=64, model_save_path='ppo_blackjack.pth'):

    input_dim = 3
    output_dim = env.action_space.n

    policy_net = PPOActorCritic(input_dim, output_dim)
    optimizer = optim.Adam(policy_net.parameters(), lr=lr)

    memory = []

    best_reward = float('-inf')
    best_model = None

    for episode in range(n_episodes):
        obs = env.reset()
        state = preprocess_state(obs)
        done = False
        episode_data = []
        episode_reward = 0

        while not done:
            state_tensor = torch.FloatTensor(state)
            logits, value = policy_net(state_tensor)
            probs = torch.softmax(logits, dim=-1)
            dist = torch.distributions.Categorical(probs)
            action = dist.sample()
            log_prob = dist.log_prob(action)

            next_obs, reward, done, _ = env.step(action.item())
            next_state = preprocess_state(next_obs)

            episode_data.append((state, action.item(), reward, log_prob.item(), value.item(), done))
            episode_reward += reward
            state = next_state

        memory.extend(episode_data)

        if len(memory) >= batch_size:
            states, actions, rewards, old_log_probs, values, dones = zip(*memory)

            returns, advantages = compute_gae(rewards, values, dones, gamma, lam)

            states_tensor = torch.FloatTensor(np.array(states))
            actions_tensor = torch.LongTensor(actions)
            old_log_probs_tensor = torch.FloatTensor(old_log_probs)
            advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

            for _ in range(epochs):
                logits, value_preds = policy_net(states_tensor)
                probs = torch.softmax(logits, dim=-1)
                dist = torch.distributions.Categorical(probs)

                new_log_probs = dist.log_prob(actions_tensor)
                ratio = torch.exp(new_log_probs - old_log_probs_tensor)

                policy_loss = -torch.min(
                    ratio * advantages,
                    torch.clamp(ratio, 1 - clip_eps, 1 + clip_eps) * advantages
                ).mean()

                value_loss = nn.MSELoss()(value_preds.squeeze(), returns)

                loss = policy_loss + 0.5 * value_loss - 0.01 * dist.entropy().mean()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            if episode_reward > best_reward:
                best_reward = episode_reward
                best_model = policy_net
                torch.save({
                    'model_state_dict': best_model.state_dict(),
                    'avg_reward': best_reward,
                    'episode': episode + 1
                }, model_save_path)
                print(f"✅ Best model saved at episode {episode+1} | Episode Reward: {best_reward:.2f}")

            memory = []

        if (episode + 1) % 1000 == 0:
            print(f"Episode {episode+1} | Last Episode Reward: {episode_reward:.2f}")

    return best_model if best_model else policy_net

In [17]:
env = BlackjackEnv(numdecks=6, natural=True)
ppo_model = train_ppo(env, n_episodes=10000)

✅ Best model saved at episode 43 | Episode Reward: -1.00
✅ Best model saved at episode 87 | Episode Reward: 1.00
✅ Best model saved at episode 348 | Episode Reward: 1.50
Episode 1000 | Last Episode Reward: -1.00
Episode 2000 | Last Episode Reward: 1.00
Episode 3000 | Last Episode Reward: 1.00
Episode 4000 | Last Episode Reward: -1.00
Episode 5000 | Last Episode Reward: -1.00
Episode 6000 | Last Episode Reward: 1.00
Episode 7000 | Last Episode Reward: 1.00
Episode 8000 | Last Episode Reward: -1.00
Episode 9000 | Last Episode Reward: -1.00
Episode 10000 | Last Episode Reward: 1.00


# Set Card Counting Strategy